In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = ''

In [ ]:
os.environ['OPENAI_API_KEY'] = ''
os.environ['COHERE_API_KEY'] = ''

In [7]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://www.geeksforgeeks.org/what-is-feature-engineering/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-template-default single single-post postid-980684 single-format-standard custom-background custom-background-white", "post-header")
        )
    ),
)


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

splits = splits[:10]

In [ ]:
# # Index
from langchain_openai import OpenAIEmbeddings
# from langchain_cohere import CohereEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    # embedding=CohereEmbeddings()
                                    embedding=OpenAIEmbeddings())


retriever = vectorstore.as_retriever()

In [8]:
print(len(splits)) 

138


In [3]:
from langchain_community.document_loaders import PyPDFLoader
pdf_load = PyPDFLoader('nsa-health-insurance-basics.pdf')
pdf_text = pdf_load.load()
pdf_text

[Document(metadata={'source': 'nsa-health-insurance-basics.pdf', 'page': 0}, page_content='1\nHealth Insurance Basics\nThis document explains key health insurance concepts that may be \nhelpful to consumers in understanding their health coverage as well \nas to consumer advocates who help individuals resolve medical \nbilling problems. This resource is not intended to describe everything that is important to \nknow about insurance. For more complete information, see the Coverage to Care resources \ndeveloped by the Centers for Medicare & Medicaid Services. \nWhat is Health Insurance and Why is it Important?\nHealth insurance is a legal entitlement to payment or reimbursement for your health care costs, generally \nunder a contract with a health insurance company.  Health insurance provides important financial \nprotection in case you have an accident or sickness. For example, health insurance may help to pay for \ndoctors’ services, medications, hospital care, and special equipment whe

In [4]:
#Dividing the data into chunks to then convert into vector embeddings for storage

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200)

document = text_splitter.split_documents(pdf_text)

document[:5]

[Document(metadata={'source': 'nsa-health-insurance-basics.pdf', 'page': 0}, page_content='1\nHealth Insurance Basics\nThis document explains key health insurance concepts that may be \nhelpful to consumers in understanding their health coverage as well \nas to consumer advocates who help individuals resolve medical \nbilling problems. This resource is not intended to describe everything that is important to \nknow about insurance. For more complete information, see the Coverage to Care resources \ndeveloped by the Centers for Medicare & Medicaid Services. \nWhat is Health Insurance and Why is it Important?\nHealth insurance is a legal entitlement to payment or reimbursement for your health care costs, generally \nunder a contract with a health insurance company.  Health insurance provides important financial \nprotection in case you have an accident or sickness. For example, health insurance may help to pay for \ndoctors’ services, medications, hospital care, and special equipment whe

In [5]:
len(document)

23

In [ ]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import  Chroma
embed = OpenAIEmbeddings()

db = Chroma.from_documents([document[0]], embed, persist_directory=None)


C:\Users\aksha\AppData\Local\Temp\ipykernel_24388\2379158842.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embed = OpenAIEmbeddings()
